## Application Steps (SQL Server Data Profile example)

#### 1. Initalize application by loading common libraries

In [ ]:
import os, sys

import dtale
from ydata_profiling import ProfileReport

import sqlalchemy as sal

## Find path of the script then find the path app_run and add it to system path ##
# path_script = os.path.abspath(__file__)
# path_app_run = os.path.dirname(os.path.dirname(path_script))
path_script_dir = os.getcwd()
path_app_run = os.path.dirname(path_script_dir)

sys.path.append(path_app_run)

## use common functions to initalize global variable and set logger ##
import utilities.common_functions as cf
import queries.sf_queries as sfq
import queries.mssql_queries as msq

loggername = 'example_mssql_connection'
logger = cf.initialize(path_app_run, loggername)

#### 2. Connect to database

In [ ]:
server = cf.gvar.mssql_server
db = cf.gvar.mssql_database
windows_auth = True

engine = cf.sal_create_enginem_ms_sql(server=server, database=db, windows_auth=windows_auth)

#### 3. Query information schema data

In [ ]:
## functions used to retreive information schema data from SQL Server ##
def list_ms_databases(_engine):
    df = cf.sal_exec_query_return_df(_engine, msq.list_dbs)
    list_dbs = df['name'].tolist()
    list_dbs.sort()
    return list_dbs

def list_ms_schemas(_engine, db):
    df = cf.sal_exec_query_return_df(_engine, msq.list_schemas.format(db_name=db))
    list_schemas = df['table_schema'].tolist()
    list_schemas.sort()
    return list_schemas

def list_ms_tables(_engine, db, schema):
    df = cf.sal_exec_query_return_df(_engine, sfq.list_tables.format(db_name=db, schema=schema))
    list_tables = df['table_name'].tolist()
    list_tables.sort()
    return list_tables

def list_ms_columns(_engine, db, schema, table):
    df = cf.sal_exec_query_return_df(_engine, sfq.get_columns.format(db_name=db, schema=schema, table=table))
    df.sort_values(by='ordinal_position', inplace=True)
    list_columns = df['column_name'].tolist()
    return list_columns


if not db:
    list_dbs = list_ms_databases(engine)
    db = list_dbs[0]

list_schemas = list_ms_schemas(engine, db)
schema = list_schemas[0]

list_tables = list_ms_tables(engine, db, schema)
table = list_tables[0]

list_columns = list_ms_columns(engine, db, schema, table)
str_columns = ', '.join(list_columns)

print(
    'database:    ' + db +
    '\nschema:      ' + schema +
    '\ntable:       ' + table +
    '\ncolumns:     ' + str_columns)

#### 4. Generate SQL statement to send

In [ ]:
def generate_sql_base_ms(_engine, db, schema, table):
    list_columns = list_ms_columns(_engine, db, schema, table)
    str_columns = ', '.join(list_columns)
    sql = f'select top 10000\n {str_columns}\nfrom  {db}.{schema}.{table}'
    return sql

sql = generate_sql_base_ms(engine, db, schema, table)
print(sql)

#### 5. Send SQL to query data and store result into pandas DataFrame

In [ ]:
def return_ms_query_df(_engine, sql):
    df = cf.sal_exec_query_return_df(_engine, sql)
    return df

df = return_ms_query_df(engine, sql)

print('\nDisplaying First 10 rows of queried result:')
df[:10]

#### 6. Profile DataFrame using dtale

In [ ]:
print('Starting D-Tale')
d = dtale.show(df, host='localhost')
d.open_browser()
dtale_running = True

#### 7. Profile DataFrame using ydata-profiling

In [ ]:
if dtale_running:
    print('D-Tale is running. Shutting down..')
    d.kill()
    dtale_running = False

def profile_data_ydata(df):
    pr = ProfileReport(df,
                    title=db + '.' + schema + '.' + table,
                    minimal=True,
                    explorative=False,
                    correlations=None,
                    infer_dtypes=False,
                    vars={
                        "num": {"low_categorical_threshold": 0},
                        "cat": {
                            "length": True,
                            "characters": False,
                            "words": False,
                            "n_obs": 10,
                        },
                    },
                    orange_mode=True)
    pr.to_notebook_iframe()

profile_data_ydata(df)